In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df0= pd.read_csv("/kaggle/input/emg-4/0.csv", header=None)
df1= pd.read_csv("/kaggle/input/emg-4/1.csv", header=None)
df2= pd.read_csv("/kaggle/input/emg-4/2.csv", header=None)
df3= pd.read_csv("/kaggle/input/emg-4/3.csv", header=None)
#rock - 0, scissors - 1, paper - 2, ok - 3

In [ ]:
print("Rock Shape: ",df0.shape,#ignore comments in this section
      "\nScissor Shape: ",df1.shape,#2910
      "\nPaper Shape: ", df2.shape,#5813
      "\nOK Shape: ",df3.shape)#8756

In [ ]:
import seaborn as sns #function for ploting frequency distribution of each sensor
import matplotlib.pyplot as plt
def plot_sensor(data,name,color):
    colors=["navy","darkmagenta","red","black"]
    fig, ax = plt.subplots(1, 8, figsize=(20,12))
    sns.set(style="white")
    sns.set(style="whitegrid")
    x=0
    for i in range(8):
        plt.suptitle(name)
        sns.distplot(data.iloc[:,x],kde=False,ax=ax[i],color=colors[color],bins=15);
        x+=1
        ax[i].set_title("Sensor_"+str(i))
            
    plt.show()


In [ ]:
plot_sensor(df0,"Rock_Data",0)
plot_sensor(df1,"Scissors_Data",1)
plot_sensor(df2,"Paper_Data",2)
plot_sensor(df3,"OK_Data",3)


In [ ]:
#sns.pairplot(df0) # plotting the correlation matrix of rock dataset
#sns.plt.show()

In [ ]:
df =pd.concat([df0,df1,df2,df3], axis=0) # combined dataframe
df

In [ ]:
plt.suptitle('Rock gesture sensor data - ploted 8 rows')
for i in range(8):
    d0=df.iloc[i]
    d0.plot()
    
plt.figure()
plt.suptitle('Scissor gesture sensor data - ploted 8 rows')
for i in range(8):
    d1=df.iloc[i+2910]
    d1.plot()
    

plt.figure()
plt.suptitle('Paper gesture sensor data - ploted 8 rows')
for i in range(8):
    d2=df.iloc[i+5813]
    d2.plot()
    
plt.figure()
plt.suptitle('Ok gesture sensor data - ploted 8 rows')
for i in range(8):
    d3=df.iloc[i+8756]
    d3.plot()

In [ ]:
list_cols = [] # rename index function on the bases of discription provided in the dataset
for i in range(8):
    for j in range(8):
        list_cols.append(f'R{i}S{j}')
list_cols.append('hand_gesture')
df.columns = list_cols
df

In [ ]:
altered_df=pd.wide_to_long(df.reset_index(), ['R0', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7'], i= ['index','hand_gesture'], j='S', sep='S')

In [ ]:
altered_df # rearranged columns  

In [ ]:
altered_df=altered_df.unstack() # rearranged columns
altered_df

In [ ]:
altered_df=altered_df.stack(level=0) # to get columns as sensors
altered_df

In [ ]:
#pd.set_option("display.max_rows",None)
#altered_df

In [ ]:
altered_df=altered_df.reset_index() # to remove repetition of serial numbers
altered_df

In [ ]:
altered_df=altered_df.drop(['index','level_2'], axis=1) # removing extra columns 

In [ ]:
col_name ="hand_gesture"
last_col = altered_df.pop(col_name)


In [ ]:
# pattern repeats after every 32nd row
colors=["forestgreen","teal","crimson","chocolate","darkred","lightseagreen","orangered","chartreuse"]
time_altered_df=altered_df.iloc[:,0:8]
time_altered_df.iloc[:256,:].plot(subplots=True,figsize=(20,10),color=colors, grid=True);

In [ ]:
altered_df.insert(8, col_name, last_col) # rearranging output column to the end of final dataframe
dataframe = altered_df
dataframe

In [ ]:
#pd.set_option("display.max_rows",None)
#dataframe

In [ ]:
m = 21 # multiplier- make sure that you do not exceed the dataframe rows 

plt.suptitle('Rock gesture sensor data - ploted 8 rows')
for i in range(8):
    d0=dataframe.drop(['hand_gesture'],axis=1).iloc[i+32*m] # 32 because each class repeats after 3x8 rows of other 3 classes
    d0.plot()
    
plt.figure()
plt.suptitle('Scissor gesture sensor data - ploted 8 rows')
for i in range(8):
    d1=dataframe.drop(['hand_gesture'],axis=1).iloc[i+8+32*m] # 8 to skip first class
    d1.plot()
    

plt.figure()
plt.suptitle('Paper gesture sensor data - ploted 8 rows')
for i in range(8):
    d2=dataframe.drop(['hand_gesture'],axis=1).iloc[i+16+32*m] # 16 to skip first two classes
    d2.plot()
    
plt.figure()
plt.suptitle('Ok gesture sensor data - ploted 8 rows')
for i in range(8):
    d3=dataframe.drop(['hand_gesture'],axis=1).iloc[i+24+32*m] # 24 to skip first 3 classes
    d3.plot()

In [ ]:
X = dataframe.drop(['hand_gesture'],axis=1) # input
y = dataframe['hand_gesture']               # output required


In [ ]:
X 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # applying KNN for classification
k_value = [] 
k_accuracy = []

for i in range(1,22):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    
    score = knn.score(X_test,y_test)
    k_value.append(i)
    k_accuracy.append(score)
for i,j in zip(k_value,k_accuracy):
    print(i,j)

#Finding k for max acc
plt.plot(range(1,22),k_accuracy,color = "blue")
plt.xlabel("K value")
plt.ylabel("Accuracy")
plt.title("Accuracy plot for KNN Classifier")
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier  # applying random forest for classification
from sklearn.metrics import accuracy_score
rf_params = None

rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix for Random Forest Classifier")
plt.show()

In [ ]:
from catboost import CatBoostClassifier  # applying cat boost for classification

cat=CatBoostClassifier(silent=True).fit(X_train,y_train)

y_pred=cat.predict(X_test)
y_pred_proba = cat.predict_proba(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix for CatBoost Classifier")
plt.show()
